In [1]:
pip install taichi pillow numpy


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:

import taichi as ti
import numpy as np
from PIL import Image


W, H = 200, 225
PORTRAIT_PATH = "/Users/kirik/Desktop/Work/Kaan.png"  # original 200x225; auto-resized to W x H

SX = 200
SY = int(np.ceil(SX * H / W))
N_SEEDS = SX * SY

BRUSH_RADIUS     = 20.0
BRUSH_STRENGTH   = 0.7      
ACTIVATION_GAIN  = 0.8      
MORPH_RATE       = 0.03     
DT               = 0.16
VISCOSITY        = 0.0005
VORT_EPS         = 0.20
PRESSURE_ITERS   = 50
FORCE_SCALE      = 18.0

SEED_MAX_SPEED   = 5.2      
SEED_REST_WEIGHT = 0.01     

try:
    ti.init(arch=ti.metal if hasattr(ti, "metal") else ti.cpu)
except Exception:
    ti.init(arch=ti.cpu)

seed_pos      = ti.Vector.field(2, ti.f32, shape=N_SEEDS)   
seed_rest_pos = ti.Vector.field(2, ti.f32, shape=N_SEEDS)   
seed_gray     = ti.field(ti.f32, shape=N_SEEDS)             
seed_tgt      = ti.field(ti.f32, shape=N_SEEDS)             
seed_act      = ti.field(ti.f32, shape=N_SEEDS)             

owner_a = ti.field(ti.i32, shape=(W, H))
owner_b = ti.field(ti.i32, shape=(W, H))

out_gray = ti.field(ti.f32, shape=(W, H))
rgb_view = ti.Vector.field(3, ti.f32, shape=(W, H))         
timg     = ti.field(ti.f32, shape=(W, H))                   

vel      = ti.Vector.field(2, ti.f32, shape=(W, H))
vel_new  = ti.Vector.field(2, ti.f32, shape=(W, H))
pressure = ti.field(ti.f32, shape=(W, H))
div      = ti.field(ti.f32, shape=(W, H))
curl     = ti.field(ti.f32, shape=(W, H))

OFFS = ti.Vector.field(2, ti.i32, shape=8)
OFFS.from_numpy(np.array([[-1,-1],[0,-1],[1,-1],[-1,0],[1,0],[-1,1],[0,1],[1,1]], dtype=np.int32))

def load_target_gray_to_canvas(path: str):
    im = Image.open(path).convert("L").resize((W, H), Image.LANCZOS)
    im = im.transpose(Image.ROTATE_180)     
    arr = (np.array(im).astype(np.float32) / 255.0)  
    return arr

def init_seeds(target_hw):
    pos   = np.zeros((N_SEEDS, 2), np.float32)
    rest  = np.zeros((N_SEEDS, 2), np.float32)
    gray  = np.ones((N_SEEDS,), np.float32)
    tgt   = np.zeros((N_SEEDS,), np.float32)
    act   = np.zeros((N_SEEDS,), np.float32)

    dx = W / SX
    dy = H / SY
    for j in range(SY):
        for i in range(SX):
            k = j * SX + i
            x = (i + 0.5) * dx
            y = (j + 0.5) * dy
            xi = min(W - 1, max(0, int(round(x))))
            yi = min(H - 1, max(0, int(round(y))))
            pos[k]  = [x, y]
            rest[k] = [x, y]
            tgt[k]  = target_hw[yi, xi]

    seed_pos.from_numpy(pos)
    seed_rest_pos.from_numpy(rest)
    seed_gray.from_numpy(gray)      
    seed_tgt.from_numpy(tgt)
    seed_act.from_numpy(act)        
    timg.from_numpy(target_hw.T)    

@ti.func
def clamp01(x: ti.f32) -> ti.f32:
    return ti.max(0.0, ti.min(1.0, x))

@ti.func
def sample_vec2(img: ti.template(), p: ti.types.vector(2, ti.f32)):
    x = clamp01(p.x / (W - 1))
    y = clamp01(p.y / (H - 1))
    fx = x * (W - 1)
    fy = y * (H - 1)
    i0 = ti.cast(ti.floor(fx), ti.i32); j0 = ti.cast(ti.floor(fy), ti.i32)
    i1 = ti.min(i0 + 1, W - 1);         j1 = ti.min(j0 + 1, H - 1)
    u = fx - i0; v = fy - j0
    return (
        img[i0, j0] * (1 - u) * (1 - v) +
        img[i1, j0] *      u  * (1 - v) +
        img[i0, j1] * (1 - u) *      v  +
        img[i1, j1] *      u  *      v
    )

@ti.kernel
def clear_owners():
    for x, y in owner_a:
        owner_a[x, y] = -1
        owner_b[x, y] = -1

@ti.kernel
def splat_ids():
    for s in range(N_SEEDS):
        x = int(seed_pos[s].x)
        y = int(seed_pos[s].y)
        if 0 <= x < W and 0 <= y < H:
            owner_a[x, y] = s

@ti.kernel
def jfa_a2b(step: ti.i32):
    for x, y in owner_a:
        best_id = owner_a[x, y]
        best_d2 = 1e30
        if best_id >= 0:
            dx = seed_pos[best_id].x - x
            dy = seed_pos[best_id].y - y
            best_d2 = dx*dx + dy*dy
        for k in ti.static(range(8)):
            nx = x + OFFS[k].x * step
            ny = y + OFFS[k].y * step
            if 0 <= nx < W and 0 <= ny < H:
                nid = owner_a[nx, ny]
                if nid >= 0:
                    ddx = seed_pos[nid].x - x
                    ddy = seed_pos[nid].y - y
                    d2 = ddx*ddx + ddy*ddy
                    if d2 < best_d2:
                        best_d2 = d2
                        best_id = nid
        owner_b[x, y] = best_id

@ti.kernel
def jfa_b2a(step: ti.i32):
    for x, y in owner_b:
        best_id = owner_b[x, y]
        best_d2 = 1e30
        if best_id >= 0:
            dx = seed_pos[best_id].x - x
            dy = seed_pos[best_id].y - y
            best_d2 = dx*dx + dy*dy
        for k in ti.static(range(8)):
            nx = x + OFFS[k].x * step
            ny = y + OFFS[k].y * step
            if 0 <= nx < W and 0 <= ny < H:
                nid = owner_b[nx, ny]
                if nid >= 0:
                    ddx = seed_pos[nid].x - x
                    ddy = seed_pos[nid].y - y
                    d2 = ddx*ddx + ddy*ddy
                    if d2 < best_d2:
                        best_d2 = d2
                        best_id = nid
        owner_a[x, y] = best_id

@ti.kernel
def shade_from_a():
    for x, y in out_gray:
        sid = owner_a[x, y]
        out_gray[x, y] = seed_gray[sid] if sid >= 0 else 1.0

@ti.kernel
def shade_from_b():
    for x, y in out_gray:
        sid = owner_b[x, y]
        out_gray[x, y] = seed_gray[sid] if sid >= 0 else 1.0

def run_jfa_and_shade():
    clear_owners()
    splat_ids()
    step = 1
    lim = max(W, H)
    while step * 2 <= lim:
        step *= 2
    flip = False
    while step >= 1:
        if not flip:
            jfa_a2b(step)
        else:
            jfa_b2a(step)
        flip = not flip
        step //= 2
    for _ in range(2):
        if not flip:
            jfa_a2b(1)
        else:
            jfa_b2a(1)
        flip = not flip
    if flip:
        shade_from_b()
    else:
        shade_from_a()


@ti.kernel
def clear_vel():
    for x, y in vel:
        vel[x, y] = ti.Vector([0.0, 0.0])

@ti.kernel
def error_force(scale: ti.f32):

    for x, y in vel:
        xm = ti.max(x-1, 0); xp = ti.min(x+1, W-1)
        ym = ti.max(y-1, 0); yp = ti.min(y+1, H-1)


        sid = owner_a[x, y] if owner_a[x, y] >= 0 else owner_b[x, y]
        act = seed_act[sid] if sid >= 0 else 0.0

        dLdx = 0.5 * ((out_gray[xp, y] - timg[xp, y]) - (out_gray[xm, y] - timg[xm, y]))
        dLdy = 0.5 * ((out_gray[x, yp] - timg[x, yp]) - (out_gray[x, ym] - timg[x, ym]))

        vel[x, y] += -scale * act * ti.Vector([dLdx, dLdy]) * DT  

@ti.kernel
def add_viscosity_once(visc: ti.f32):
    for x, y in vel:
        s = ti.Vector([0.0, 0.0]); c = 0
        if x > 0:   s += vel[x-1, y]; c += 1
        if x < W-1: s += vel[x+1, y]; c += 1
        if y > 0:   s += vel[x, y-1]; c += 1
        if y < H-1: s += vel[x, y+1]; c += 1
        vel_new[x, y] = (vel[x, y] + visc * s) / (1.0 + visc * c)
    for x, y in vel:
        vel[x, y] = vel_new[x, y]

def add_viscosity(n, visc):
    for _ in range(n):
        add_viscosity_once(visc)

@ti.kernel
def compute_curl():
    for x, y in curl:
        dx = 0.5 * (vel[min(x+1, W-1), y].y - vel[max(x-1, 0), y].y)
        dy = 0.5 * (vel[x, min(y+1, H-1)].x - vel[x, max(y-1, 0)].x)
        curl[x, y] = dx - dy

@ti.kernel
def vorticity_confinement(eps: ti.f32):
    for x, y in vel:
        xm = max(x-1, 0); xp = min(x+1, W-1)
        ym = max(y-1, 0); yp = min(y+1, H-1)
        Nx = 0.5 * (abs(curl[xp, y]) - abs(curl[xm, y]))
        Ny = 0.5 * (abs(curl[x, yp]) - abs(curl[x, ym]))
        nlen = ti.sqrt(Nx*Nx + Ny*Ny) + 1e-5
        n = ti.Vector([Nx, Ny]) / nlen
        vel[x, y] += eps * ti.Vector([n.y, -n.x]) * curl[x, y] * DT

@ti.kernel
def divergence():
    for x, y in div:
        vL = vel[max(x-1, 0), y].x
        vR = vel[min(x+1, W-1), y].x
        vB = vel[x, max(y-1, 0)].y
        vT = vel[x, min(y+1, H-1)].y
        div[x, y] = -0.5 * ((vR - vL) + (vT - vB))

@ti.kernel
def pressure_jacobi_once():
    for x, y in pressure:
        pL = pressure[max(x-1, 0), y]
        pR = pressure[min(x+1, W-1), y]
        pB = pressure[x, max(y-1, 0)]
        pT = pressure[x, min(y+1, H-1)]
        pressure[x, y] = 0.25 * (pL + pR + pB + pT + div[x, y])

def pressure_solve(iters: int):
    pressure.fill(0.0)
    for _ in range(iters):
        pressure_jacobi_once()

@ti.kernel
def subtract_pressure_grad():
    for x, y in vel:
        pL = pressure[max(x-1, 0), y]
        pR = pressure[min(x+1, W-1), y]
        pB = pressure[x, max(y-1, 0)]
        pT = pressure[x, min(y+1, H-1)]
        gradp = 0.5 * ti.Vector([pR - pL, pT - pB])
        vel[x, y] -= gradp

@ti.kernel
def advect_velocity():
    for x, y in vel:
        p = ti.Vector([ti.cast(x, ti.f32), ti.cast(y, ti.f32)])
        back = p - DT * sample_vec2(vel, p)
        vel_new[x, y] = sample_vec2(vel, back)
    for x, y in vel:
        vel[x, y] = vel_new[x, y]

@ti.kernel
def advect_seeds(dt: ti.f32, vmax: ti.f32, rest_w: ti.f32):
    for s in range(N_SEEDS):
        p = seed_pos[s]
        v = sample_vec2(vel, p)
        sp = ti.sqrt(v.x*v.x + v.y*v.y)
        if sp > vmax:
            v *= vmax / (sp + 1e-6)
        p += v * dt

        p = p * (1.0 - rest_w) + seed_rest_pos[s] * rest_w

        p.x = clamp01(p.x / (W - 1)) * (W - 1)
        p.y = clamp01(p.y / (H - 1)) * (H - 1)
        seed_pos[s] = p

@ti.kernel
def morph_seed_grays(rate: ti.f32):
    for s in range(N_SEEDS):
        a = seed_act[s]
        if a > 1e-4:
            seed_gray[s] += (seed_tgt[s] - seed_gray[s]) * (rate * a)

@ti.kernel
def paint_seeds(mx: ti.f32, my: ti.f32, radius: ti.f32, color_strength: ti.f32, act_gain: ti.f32):
    for s in range(N_SEEDS):
        d = (seed_pos[s] - ti.Vector([mx, my])).norm()
        if d < radius:
            w = 1.0 - d / radius
            
            k = w * color_strength
            seed_gray[s] = seed_gray[s] * (1.0 - k)
            
            seed_act[s] = ti.min(1.0, seed_act[s] + w * act_gain)

@ti.kernel
def pack_rgb_from_gray():
    for x, y in rgb_view:
        g = out_gray[x, y]
        rgb_view[x, y] = ti.Vector([g, g, g])

def main():
    target = load_target_gray_to_canvas(PORTRAIT_PATH)  
    init_seeds(target)

    window = ti.ui.Window("Kaanify 1.0", (W, H))
    canvas = window.get_canvas()
    frame = 0
    print(f"[init] {W}x{H}, seeds {SX}x{SY} = {N_SEEDS}")
    print("[controls] LMB draw | C clear | S save")

    clear_vel()
    out_gray.fill(1.0)  

    while window.running:
        if window.is_pressed(ti.ui.LMB):
            u = window.get_cursor_pos()   
            mx = u[0] * (W - 1)
            my = u[1] * (H - 1)          
            paint_seeds(mx, my, BRUSH_RADIUS, BRUSH_STRENGTH, ACTICATION_GAIN:=ACTIVATION_GAIN)

        if window.get_event(ti.ui.PRESS):
            if window.event.key in ['c', 'C']:
                seed_gray.from_numpy(np.ones((N_SEEDS,), np.float32))
                seed_act.from_numpy(np.zeros((N_SEEDS,), np.float32))
                clear_vel()
                out_gray.fill(1.0)
                print("[info] cleared to white & deactivated")
            elif window.event.key in ['s', 'S']:
                img = (np.clip(out_gray.to_numpy().T * 255, 0, 255)).astype(np.uint8)
                Image.fromarray(img, mode='L').save(f"frame_{frame:04d}.png")
                print(f"[save] frame_{frame:04d}.png")

        morph_seed_grays(MORPH_RATE)

        run_jfa_and_shade()

        error_force(FORCE_SCALE)        
        add_viscosity(10, VISCOSITY)
        compute_curl()
        vorticity_confinement(VORT_EPS)
        advect_velocity()
        divergence()
        pressure_solve(PRESSURE_ITERS)
        subtract_pressure_grad()
        advect_seeds(DT, SEED_MAX_SPEED, SEED_REST_WEIGHT)

 
        pack_rgb_from_gray()
        canvas.set_image(rgb_view)
        window.show()
        frame += 1

if __name__ == "__main__":
    main()

[Taichi] version 1.7.4, llvm 15.0.7, commit b4b956fd, osx, python 3.12.2


[I 10/14/25 00:22:53.809 34275465] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


[Taichi] Starting on arch=metal


2025-10-14 00:22:54.701 Python[2394:34275465] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


[init] 200x225, seeds 200x225 = 45000
[controls] LMB draw | C clear | S save


: 


# Mathematical Principles Behind `Kaanify`

This simulation merges **computational geometry**, **fluid dynamics**, and **optimization** into a dynamic morphing system that reconstructs a portrait from a blank canvas.

## 1. Overview

The algorithm evolves a field of *seeds* and a *fluid velocity field* to minimize the image energy:
$$
E = \int_\Omega \big(I(x,y) - T(x,y)\big)^2 \, dx\,dy
$$
where \(I(x,y)\) is the current generated image and \(T(x,y)\) is the target portrait.

## 2. Jump Flood Algorithm (JFA)

JFA approximates a **Voronoi diagram**: each pixel \(p\) takes the nearest seed \(s_i\) by Euclidean distance
$$
s^*(p) = \arg\min_{s_i} \,\|\,p - \mathbf{x}_{s_i}\,\|^2.
$$
The pixel’s gray equals the owning seed’s gray. This is a fast multi-pass approximation of the Euclidean distance transform.

## 3. Morphing (Color Relaxation)

Each seed \(i\) has current gray \(g_i(t)\), a target gray \(g_i^*\), and activation \(a_i \in [0,1]\).  
We update by explicit Euler (relaxation):

$$
g_i(t+\Delta t) = g_i(t) + a_i\,\lambda\,\big(g_i^* - g_i(t)\big),
$$

which discretizes the ODE

$$
\frac{dg_i}{dt} = a_i \lambda \big(g_i^* - g_i\big).
$$

Active seeds exponentially approach their targets.

## 4. Activation Diffusion (Optional)

If activation spreads, it follows the **heat equation** in discrete form:
$$
a'(x,y) \;=\; a(x,y) + D\,\nabla^2 a(x,y), \qquad
\frac{\partial a}{\partial t} = D\,\nabla^2 a.
$$

## 5. Fluid Dynamics (Navier–Stokes Approximation)

We simulate an incompressible 2D velocity field \(\mathbf{v}(x,y,t)\) with viscosity \(\nu\) and pressure \(p\):
$$
\begin{cases}
\displaystyle \frac{\partial \mathbf{v}}{\partial t} + (\mathbf{v}\cdot\nabla)\mathbf{v}
= -\nabla p + \nu \nabla^2 \mathbf{v} + \mathbf{f}, \\[6pt]
\displaystyle \nabla \cdot \mathbf{v} = 0.
\end{cases}
$$

### Error‑Driven Forcing
The external force reduces image error by following the gradient of the difference:
$$
\mathbf{f}(x,y) \;=\; -\,k\,A(x,y)\,\nabla\!\big(I(x,y) - T(x,y)\big),
$$
where \(A(x,y)\) is an activation mask (seeds you’ve “painted”).

## 6. Pressure Projection (Poisson Solve)

To enforce incompressibility, we solve the **Poisson equation** for pressure and project:
$$
\nabla^2 p \;=\; \nabla\cdot\mathbf{v}, \qquad
\mathbf{v} \leftarrow \mathbf{v} - \nabla p,
$$
implemented via Jacobi iterations in the code.

## 7. Vorticity Confinement

To restore swirl lost to numerical diffusion, we add
$$
\mathbf{v} \leftarrow \mathbf{v} + \epsilon \,\big(\nabla |\omega| \times \hat{z}\big)\,\omega,
\qquad \omega \;=\; \nabla\times\mathbf{v}.
$$

## 8. Seed Advection + Spring Constraint

Seeds move with the flow and are softly pulled toward a rest grid (to avoid holes):
$$
\mathbf{x}_i \leftarrow (1-w)\Big(\mathbf{x}_i + \mathbf{v}(\mathbf{x}_i)\,\Delta t\Big) + w\,\mathbf{x}_i^{\text{rest}}.
$$

## 9. Numerical Building Blocks

- **JFA / Voronoi ownership:** multi-pass nearest‑seed propagation  
- **Advection:** semi‑Lagrangian backtrace sampling  
- **Diffusion:** discrete Laplacian smoothing  
- **Pressure:** Jacobi iterations for Poisson equation  
- **Morphing:** explicit Euler relaxation

## 10. One‑Line Summary

*The system morphs a blank canvas into the portrait by performing a fluid‑driven gradient descent of the image error over a Voronoi‑tessellated domain.*
